In [1]:
#colab 을 이용한 실행시
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchtext.data as data
import torchtext.datasets as datasets
import pickle

In [0]:
class CNN_Text(nn.Module):
    
    def __init__(self, embed_num, class_num):
        super(CNN_Text, self).__init__()
        V = embed_num 
        C = class_num
        Co = [50, 100, 50] #args.kernel_num
        Ks = [2,3,4]

        self.embed = nn.Embedding(V, 100) #embedding 100차원으로 
        self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, (K, 100)) for K in Ks])
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(len(Ks)*Co, C)

    def forward(self, x):
        x = self.embed(x)  # (N, W, D)
        x = x.unsqueeze(1)  # (N, Ci, W, D)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)
        x = torch.cat(x, 1)
        x = self.dropout(x)  # (N, len(Ks)*Co)
        logit = self.fc1(x)  # (N, C)
        return logit

In [30]:
Co = [50, 100, 50]
Ks = [2, 3, 4]
nn.ModuleList([nn.Conv2d(1, Co[0], (K, 100)) for K in Ks])

ModuleList(
  (0): Conv2d(1, 50, kernel_size=(2, 100), stride=(1, 1))
  (1): Conv2d(1, 50, kernel_size=(3, 100), stride=(1, 1))
  (2): Conv2d(1, 50, kernel_size=(4, 100), stride=(1, 1))
)

In [0]:
class mydataset(data.Dataset):
    @staticmethod
    def sort_key(ex):
        return len(ex.text)
    def __init__(self, text_field, label_field, path=None, examples=None, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        if examples is None:
            path = self.dirname if path is None else path
            examples = []
            for i,line in enumerate(open(path,'r',encoding='utf-8')):
                if i==0:
                    continue
                line = line.strip().split('\t')
                txt = line[1].split(' ')
                #txt= [ d.split('/')[0] for d in line[1].split(' ') ]
                
                examples += [ data.Example.fromlist( [ txt, line[2]],fields ) ]
        super(mydataset, self).__init__(examples, fields, **kwargs)

In [5]:
text_field = data.Field(batch_first = True, fix_length = 20 ) # fix_length: 한 문장의 max 토큰수 길이
label_field = data.Field(sequential=False, batch_first = True, unk_token = None )

train_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/dongguk_nlp/nsm/small_ratings_train_tok.txt') #텍스트파일 형태소구분 띄어쓰기한 파일 

test_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/dongguk_nlp/nsm/small_ratings_test_tok.txt')

text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

train_iter, test_iter = data.Iterator.splits(
                            (train_data, test_data), 
                            batch_sizes=(100, 1), repeat=False)#, device= 'cuda')
len(text_field.vocab)

21893

In [25]:
cnn = CNN_Text(len(text_field.vocab),2)
optimizer = torch.optim.Adam(cnn.parameters()) #kernel size ==filter size 2-> 2단어 3->3단어 
cnn.train()


TypeError: ignored

In [8]:
%%time
# 실행 전에 구글 드라이브 Colab Notebooks 폴더에 model 폴더 생성
for epoch in range(10):
    z=0
    totalloss = 0
    for batch in train_iter:
        optimizer.zero_grad()
        
        txt = batch.text
        label = batch.label
        #print (txt.size())
        pred = cnn(txt)
        #print(pred.size(), label.size())
        #print(label)
        loss = F.cross_entropy(pred, label)
        totalloss += loss.data
        
        loss.backward()
        optimizer.step()
        #print(data,label)
    print(epoch,'epoch')    
    print(totalloss)

torch.save(cnn,'/content/gdrive/My Drive/Colab Notebooks/cnn_model.pt')

0 epoch
tensor(36.0768)
1 epoch
tensor(27.9582)
2 epoch
tensor(21.2576)
3 epoch
tensor(16.2742)
4 epoch
tensor(12.0669)
5 epoch
tensor(9.7048)
6 epoch
tensor(8.3554)
7 epoch
tensor(6.4208)
8 epoch
tensor(5.4311)
9 epoch
tensor(4.3160)
CPU times: user 1min 5s, sys: 1.82 s, total: 1min 7s
Wall time: 1min 7s


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CNN_Text. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleList. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.

In [9]:
%%time
cnn.eval()
correct = 0
incorrect = 0
for batch in test_iter:
    txt = batch.text
    label = batch.label
    
    pred = cnn(txt)
    _,ans = torch.max(pred,dim=1)
    
    if ans.data[0] == label.data[0]:        
        correct += 1    
    else:
        incorrect += 1
    
print ('correct : ', correct)
print ('incorrect : ', incorrect)
print ()

correct :  86
incorrect :  14

CPU times: user 85 ms, sys: 5.98 ms, total: 91 ms
Wall time: 97.2 ms
